In [1]:
# Import dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
import pandas as pd
import tensorflow as tf

# Import out input dataset
attrition_df = pd.read_csv("HR-Employee-Attrition.csv")
attrition_df.head()

,Age,Attrition,BusinessTravel,DailyRate,Department,DistanceFromHome,Education,EducationField,EmployeeCount,EmployeeNumber,...,RelationshipSatisfaction,StandardHours,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
0,41,Yes,Travel_Rarely,1102,Sales,1,2,Life Sciences,1,1,...,1,80,0,8,0,1,6,4,0,5
1,49,No,Travel_Frequently,279,Research & Development,8,1,Life Sciences,1,2,...,4,80,1,10,3,3,10,7,1,7
2,37,Yes,Travel_Rarely,1373,Research & Development,2,2,Other,1,4,...,2,80,0,7,3,3,0,0,0,0
3,33,No,Travel_Frequently,1392,Research & Development,3,4,Life Sciences,1,5,...,3,80,0,8,3,3,8,7,3,0
4,27,No,Travel_Rarely,591,Research & Development,2,1,Medical,1,7,...,4,80,1,6,3,3,2,2,2,2


In [3]:
# Generate our categorical variable list
attrition_cat = attrition_df.dtypes[attrition_df.dtypes == "object"].index.tolist()
attrition_cat

['Attrition',
 'BusinessTravel',
 'Department',
 'EducationField',
 'Gender',
 'JobRole',
 'MaritalStatus',
 'Over18',
 'OverTime']

In [5]:
# Make sure non of the categorical variables have more than 10 unique values and require bucketing 

# Check the number of unique values in each column 
attrition_df[attrition_cat].nunique()

Attrition         2
BusinessTravel    3
Department        3
EducationField    6
Gender            2
JobRole           9
MaritalStatus     3
Over18            1
OverTime          2
dtype: int64

In [6]:
# Create a OneHotEncoder instance
enc = OneHotEncoder(sparse=False)

# Fit and transform the OneHoteEncoder using the categorical variable list
encode_df = pd.DataFrame(enc.fit_transform(attrition_df[attrition_cat]))

# Add the encoded variable names to the DataFrame
encode_df.columns = enc.get_feature_names(attrition_cat)
encode_df.head()

,Attrition_No,Attrition_Yes,BusinessTravel_Non-Travel,BusinessTravel_Travel_Frequently,BusinessTravel_Travel_Rarely,Department_Human Resources,Department_Research & Development,Department_Sales,EducationField_Human Resources,EducationField_Life Sciences,...,JobRole_Research Director,JobRole_Research Scientist,JobRole_Sales Executive,JobRole_Sales Representative,MaritalStatus_Divorced,MaritalStatus_Married,MaritalStatus_Single,Over18_Y,OverTime_No,OverTime_Yes
0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0
1,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0
2,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0
3,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0
4,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0


In [7]:
# to replace these columns, use merge and drop methods 

# Merge one-hot encoded features and drop the originals
attrition_df = attrition_df.merge(encode_df, left_index=True, right_index=True)
attrition_df = attrition_df.drop(attrition_cat, 1)
attrition_df.head()

,Age,DailyRate,DistanceFromHome,Education,EmployeeCount,EmployeeNumber,EnvironmentSatisfaction,HourlyRate,JobInvolvement,JobLevel,...,JobRole_Research Director,JobRole_Research Scientist,JobRole_Sales Executive,JobRole_Sales Representative,MaritalStatus_Divorced,MaritalStatus_Married,MaritalStatus_Single,Over18_Y,OverTime_No,OverTime_Yes
0,41,1102,1,2,1,1,2,94,3,2,...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0
1,49,279,8,1,1,2,3,61,2,2,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0
2,37,1373,2,2,1,4,4,92,2,1,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0
3,33,1392,3,4,1,5,4,56,3,1,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0
4,27,591,2,1,1,7,1,40,3,1,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0


In [8]:
# Split our preprocessed data into our features and target arrays
y = attrition_df["Attrition_Yes"].values
X = attrition_df.drop(["Attrition_Yes", "Attrition_No"], 1).values

# Split the preprocessed data into a traiing and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=78)

In [9]:
# Create a StandardScaler instance
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

### 19.4.4: Real Data, Real Practice Deep Learning Model Design 

In [10]:
# Add layers to the "Deep" Neural Network model (input layer, hidden layer and output laayer)
# input layer: # of variable in Df, hidden layer: 2 layers and use Keras Dense class to create 
# additional layer. Output layer

# Defie the model - deep neural net
number_input_features = len(X_train[0])
hidden_nodes_layer1 = 8
hidden_nodes_layer2 = 5

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim= number_input_features, 
                         activation="relu"))

# Second hidden layer
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model 
nn.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 8)                 448       
                                                                 
 dense_1 (Dense)             (None, 5)                 45        
                                                                 
 dense_2 (Dense)             (None, 1)                 6         
                                                                 
Total params: 499
Trainable params: 499
Non-trainable params: 0
_________________________________________________________________


In [11]:
# comple the model 
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"]) 

In [12]:
# Train the model 
fit_model = nn.fit(X_train_scaled, y_train, epochs=100)

Epoch 1/100
35/35 [==============================] - 0s 997us/step - loss: 0.6031 - accuracy: 0.7223
Epoch 2/100
35/35 [==============================] - 0s 909us/step - loss: 0.5178 - accuracy: 0.8022
Epoch 3/100
35/35 [==============================] - 0s 1ms/step - loss: 0.4680 - accuracy: 0.8230
Epoch 4/100
35/35 [==============================] - 0s 1ms/step - loss: 0.4368 - accuracy: 0.8258
Epoch 5/100
35/35 [==============================] - 0s 1ms/step - loss: 0.4169 - accuracy: 0.8276
Epoch 6/100
35/35 [==============================] - 0s 968us/step - loss: 0.4020 - accuracy: 0.8267
Epoch 7/100
35/35 [==============================] - 0s 1ms/step - loss: 0.3900 - accuracy: 0.8276
Epoch 8/100
35/35 [==============================] - 0s 1ms/step - loss: 0.3797 - accuracy: 0.8303
Epoch 9/100
35/35 [==============================] - 0s 1ms/step - loss: 0.3711 - accuracy: 0.8339
Epoch 10/100
35/35 [==============================] - 0s 1ms/step - loss: 0.3627 - accuracy: 0.8358
Epo

35/35 [==============================] - 0s 939us/step - loss: 0.1636 - accuracy: 0.9401
Epoch 83/100
35/35 [==============================] - 0s 909us/step - loss: 0.1623 - accuracy: 0.9401
Epoch 84/100
35/35 [==============================] - 0s 851us/step - loss: 0.1609 - accuracy: 0.9419
Epoch 85/100
35/35 [==============================] - 0s 939us/step - loss: 0.1594 - accuracy: 0.9428
Epoch 86/100
35/35 [==============================] - 0s 909us/step - loss: 0.1577 - accuracy: 0.9428
Epoch 87/100
35/35 [==============================] - 0s 909us/step - loss: 0.1572 - accuracy: 0.9446
Epoch 88/100
35/35 [==============================] - 0s 880us/step - loss: 0.1560 - accuracy: 0.9419
Epoch 89/100
35/35 [==============================] - 0s 997us/step - loss: 0.1543 - accuracy: 0.9437
Epoch 90/100
35/35 [==============================] - 0s 1ms/step - loss: 0.1533 - accuracy: 0.9419
Epoch 91/100
35/35 [==============================] - 0s 1ms/step - loss: 0.1517 - accuracy: 0.94

In [13]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled, y_test, verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

12/12 - 0s - loss: 0.4217 - accuracy: 0.8587 - 122ms/epoch - 10ms/step
Loss: 0.42174944281578064, Accuracy: 0.8586956262588501


#### Checkpoint 

compile and train model using checkpoints. 

In [14]:
# Import checkpoint dependencies
import os
from tensorflow.keras.callbacks import ModelCheckpoint

# Define the checkpoint path and filenames
os.makedirs("checkpoints/", exist_ok=True)
checkpoints_path = "checkpoints/weights.{epoch:02d}.hdf5"

In [15]:
# Compile the model 
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# Create a callback that saves the model's weights every epoch
cp_callback = ModelCheckpoint(filepath=checkpoints_path,
                             verbose=1, 
                             save_weights_only=True, 
                             save_freq="epoch")

# Train the model
fit_model = nn.fit(X_train_scaled, y_train, epochs=100, callbacks=[cp_callback])

# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled, y_test, verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

Epoch 1/100
 1/35 [..............................] - ETA: 10s - loss: 0.0628 - accuracy: 1.0000
Epoch 1: saving model to checkpoints\weights.01.hdf5
35/35 [==============================] - 0s 2ms/step - loss: 0.1427 - accuracy: 0.9446
Epoch 2/100
 1/35 [..............................] - ETA: 0s - loss: 0.1066 - accuracy: 0.9375
Epoch 2: saving model to checkpoints\weights.02.hdf5
35/35 [==============================] - 0s 1ms/step - loss: 0.1401 - accuracy: 0.9483
Epoch 3/100
 1/35 [..............................] - ETA: 0s - loss: 0.2013 - accuracy: 0.9062
Epoch 3: saving model to checkpoints\weights.03.hdf5
35/35 [==============================] - 0s 1ms/step - loss: 0.1384 - accuracy: 0.9483
Epoch 4/100
 1/35 [..............................] - ETA: 0s - loss: 0.1457 - accuracy: 0.9375
Epoch 4: saving model to checkpoints\weights.04.hdf5
35/35 [==============================] - 0s 1ms/step - loss: 0.1379 - accuracy: 0.9519
Epoch 5/100
 1/35 [..............................] - ETA: 0

Epoch 36/100
 1/35 [..............................] - ETA: 0s - loss: 0.0784 - accuracy: 0.9688
Epoch 36: saving model to checkpoints\weights.36.hdf5
35/35 [==============================] - 0s 1ms/step - loss: 0.1009 - accuracy: 0.9637
Epoch 37/100
34/35 [============================>.] - ETA: 0s - loss: 0.1001 - accuracy: 0.9688
Epoch 37: saving model to checkpoints\weights.37.hdf5
35/35 [==============================] - 0s 2ms/step - loss: 0.1004 - accuracy: 0.9691
Epoch 38/100
 1/35 [..............................] - ETA: 0s - loss: 0.1306 - accuracy: 0.9375
Epoch 38: saving model to checkpoints\weights.38.hdf5
35/35 [==============================] - 0s 1ms/step - loss: 0.0981 - accuracy: 0.9673
Epoch 39/100
 1/35 [..............................] - ETA: 0s - loss: 0.0373 - accuracy: 1.0000
Epoch 39: saving model to checkpoints\weights.39.hdf5
35/35 [==============================] - 0s 1ms/step - loss: 0.0983 - accuracy: 0.9664
Epoch 40/100
 1/35 [..............................] 

35/35 [==============================] - 0s 1ms/step - loss: 0.0766 - accuracy: 0.9737
Epoch 71/100
 1/35 [..............................] - ETA: 0s - loss: 0.0195 - accuracy: 1.0000
Epoch 71: saving model to checkpoints\weights.71.hdf5
35/35 [==============================] - 0s 1ms/step - loss: 0.0762 - accuracy: 0.9737
Epoch 72/100
35/35 [==============================] - ETA: 0s - loss: 0.0760 - accuracy: 0.9746
Epoch 72: saving model to checkpoints\weights.72.hdf5
35/35 [==============================] - 0s 2ms/step - loss: 0.0760 - accuracy: 0.9746
Epoch 73/100
 1/35 [..............................] - ETA: 0s - loss: 0.0930 - accuracy: 0.9375
Epoch 73: saving model to checkpoints\weights.73.hdf5
35/35 [==============================] - 0s 1ms/step - loss: 0.0756 - accuracy: 0.9737
Epoch 74/100
 1/35 [..............................] - ETA: 0s - loss: 0.1043 - accuracy: 0.9375
Epoch 74: saving model to checkpoints\weights.74.hdf5
35/35 [==============================] - 0s 1ms/step

To test the above checkpoint functionality, define another deep learning model, but restore the weights using the checkpoints rather than training the model 

In [18]:
# Create another deep learning model 
# Define the model - deep neural net 
number_input_features= len(X_train_scaled[0])
hidden_nodes_layer1= 8
hidden_nodes_layer2 = 5

nn_new = tf.keras.models.Sequential()

# First hidden layer
nn_new.add(tf.keras.layers.Dense(units=hidden_nodes_layer1, 
                                input_dim= number_input_features,
                                 activation="relu"))

# Second hidden layer
nn_new.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Output layer
nn_new.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# compile the model
nn_new.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

# Restore the model weights
nn_new.load_weights("checkpoints/weights.100.hdf5")

# Evaluate the model using test data
model_loss, model_accuracy = nn_new.evaluate(X_test_scaled, y_test, verbose=2)
print(f"Loss: {model_loss}, Accuracy {model_accuracy}")

12/12 - 0s - loss: 0.7650 - accuracy: 0.8424 - 129ms/epoch - 11ms/step
Loss: 0.7649773359298706, Accuracy 0.842391312122345


### Exporting and Importing Entire model 
Save model weights during training using Checkpoints. Use Sequential model's save method to export entire model to HDF5. Once saved, import the exact same trained model to the environment using load_model method using Keras

In [19]:
# Export out model to HDF5
nn_new.save("trained_attrition.h5")

In [20]:
# Import the model to a new object
nn_imported = tf.keras.models.load_model("trained_attrition.h5")

In [21]:
# Test and evaluate the completed model using test data
model_loss, model_accuracy = nn_imported.evaluate(X_test_scaled, y_test, verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

12/12 - 0s - loss: 0.7650 - accuracy: 0.8424 - 95ms/epoch - 8ms/step
Loss: 0.7649773359298706, Accuracy: 0.842391312122345
